In [4]:
import numpy as np
from scipy.stats import norm
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import check_X_y, check_array

class OrdinalRegression(BaseEstimator, RegressorMixin):
    def __init__(self, base_learner=DecisionTreeRegressor(), n_classes=3, max_iter=100, tol=1e-4):
        self.base_learner = base_learner
        self.n_classes = n_classes
        self.max_iter = max_iter
        self.tol = tol

    def _initialize_thresholds(self, y):
        # Calculate the initial threshold vector
        n_samples = len(y)
        P = np.array([np.sum(y == i) for i in range(self.n_classes)]) / n_samples
        self.thresholds_ = norm.ppf(np.cumsum(P[:-1]))

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)

        # Initialize the model
        self._initialize_thresholds(y)
        self.g_ = np.zeros(X.shape[0])
        self.models_ = []

        for iteration in range(self.max_iter):
            print(self.thresholds_)
            
            prev_g = self.g_.copy()
            
            # Gradient boosting step: Fit the base learner to the pseudo-residuals
            residuals = self._compute_pseudo_residuals(X, y)
            model = self.base_learner.fit(X, residuals)
            self.models_.append(model)
            self.g_ += model.predict(X)

            # Check for convergence
            if np.mean((self.g_ - prev_g) ** 2) < self.tol:
                break

        return self

    def _compute_pseudo_residuals(self, X, y):
        residuals = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            mu = self.g_[i]
            lower = self.thresholds_[y[i] - 1] if y[i] > 0 else -np.inf
            upper = self.thresholds_[y[i]] if y[i] < self.n_classes - 1 else np.inf

            f_lower = norm.pdf(lower - mu)
            f_upper = norm.pdf(upper - mu)
            F_lower = norm.cdf(lower - mu)
            F_upper = norm.cdf(upper - mu)
            
            residuals[i] = (f_lower - f_upper) / (F_upper - F_lower)
        
        return residuals

    def predict(self, X):
        check_array(X)

        # Compute the latent variable g(x)
        g = np.zeros(X.shape[0])
        for model in self.models_:
            g += model.predict(X)
        
        # Assign the ordinal class based on the threshold
        y_pred = np.digitize(g, self.thresholds_)
        return y_pred

    def score(self, X, y):
        from sklearn.metrics import accuracy_score
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)


In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create a sample dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, n_informative=5, random_state=0)
y = np.digitize(y, bins=[0.33, 0.66])  # Convert to ordinal classes

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = OrdinalRegression(base_learner=DecisionTreeRegressor(max_depth=3), n_classes=3, max_iter=10)
model.fit(X_train, y_train)

# Evaluate the model
print("Accuracy:", model.score(X_test, y_test))


[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
[-0.42614801 -0.42614801]
Accuracy: 0.675


In [3]:
model.thresholds_

array([-0.42614801, -0.42614801])